In [1]:
import numpy as np
import pandas as pd
import statsrat as sr
from statsrat import rw
from statsrat import expr

In [2]:
# CompAct (with only elemental features); Model 4 from Paskewitz and Jones (2020).

CompAct = rw.model(name = 'CompAct',
                   pred = rw.pred.identity,
                   fbase = rw.fbase.elem,
                   fweight = rw.fweight.from_aux_norm,
                   lrate = rw.lrate.from_aux_norm,
                   drate = rw.drate.zero,
                   aux = rw.aux.gradcomp)

In [3]:
# Le Pelley and McLaren 2003 (learned predictiveness)
# Test responses were really rating scales for both response options instead of choices.

design = expr.schedule(resp_type = 'choice',
                      stages = {'training': expr.stage(
                                        x_pn = [['a', 'v'], ['b', 'v'], ['a', 'w'], ['b', 'w'], ['c', 'x'], ['d', 'x'], ['c', 'y'], ['d', 'y']],
                                        y = 4*[['cat1'], ['cat2']],
                                        y_psb = ['cat1', 'cat2'],
                                        n_rep = 14),
                                  'transfer': expr.stage(
                                        x_pn = [['a', 'x'], ['b', 'y'], ['c', 'v'], ['d', 'w'], ['e', 'f'], ['g', 'h'], ['i', 'j'], ['k', 'l']],
                                        y = 4*[['cat3'], ['cat4']],
                                        y_psb = ['cat3', 'cat4'],
                                        n_rep = 4),
                                  'test': expr.stage(
                                        x_pn = [['a', 'c'], ['b', 'd'], ['v', 'x'], ['w', 'y'], ['e', 'h'], ['f', 'g'], ['i', 'j'], ['k', 'l']],
                                        y_psb = ['cat3', 'cat4'],
                                        lrn = False,
                                        n_rep = 1)})

rel_irl = expr.oat(schedule_pos = ['design'],
                  behav_score_pos = expr.behav_score(stage = 'test',
                                                    trial_pos = ['a.c -> nothing', 'b.d -> nothing'],
                                                    trial_neg = ['v.x -> nothing', 'w.y -> nothing'],
                                                    resp_pos = ['cat3', 'cat4'],
                                                    resp_neg = ['cat3', 'cat4']))

lrn_pred = expr.experiment(schedules = {'design': design},
                           oats = {'rel_irl': rel_irl})

In [4]:
# Create synthetic data

sim_data = sr.make_sim_data(model = CompAct, experiment = lrn_pred)
n_t = sim_data['ds']['t'].shape[0]
resp_names = pd.Series(['a', 's', 'd', 'f'])
cue_names = pd.Series(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'V', 'W', 'X', 'Y'])

for ident in sim_data['ds']['ident'].values:
    bar = sim_data['ds'].loc[{'ident': ident}]
    resp = np.array(n_t*[''])
    left_stim = np.array(n_t*[''])
    right_stim = np.array(n_t*[''])
    confidence_rating = np.zeros(n_t, dtype = 'float')
    for t in range(n_t):
        resp[t] = resp_names.loc[bar['b'].loc[{'t': t}].values == 1.0].values[0]
        cues = cue_names.loc[bar['x'].loc[{'t': t}].values == 1.0].values
        if t%2 == 0:
            left_stim[t] = cues[0]
            right_stim[t] = cues[1]
        else:
            left_stim[t] = cues[1]
            right_stim[t] = cues[0]
        # Create synthetic confidence scores (this should really be based on a y_hat difference)
        if 'nothing' in bar['trial_name'].values[t]:
            choice_y_hat = bar['y_hat'].loc[{'t': t, 'y_name': bar['b_name'].loc[{'t': t}]}].values
            confidence_rating[t] = np.round(10*choice_y_hat)
    quz = pd.DataFrame({'key_press': resp,
                        'left_stim': left_stim,
                        'right_stim': right_stim,
                        'confidence_rating': confidence_rating,
                        'subject_id': ident})
    quz.to_csv('data/' + ident + '_data.csv')

In [5]:
print(quz)

    key_press left_stim right_stim  confidence_rating subject_id
0           a         D          Y                0.0       sim9
1           s         V          A                0.0       sim9
2           s         B          W                0.0       sim9
3           s         X          C                0.0       sim9
4           a         A          W                0.0       sim9
..        ...       ...        ...                ...        ...
147         f         G          F                5.0       sim9
148         f         V          X                0.0       sim9
149         f         L          K               10.0       sim9
150         f         B          D               15.0       sim9
151         d         J          I               10.0       sim9

[152 rows x 5 columns]
